In [8]:
!git clone https://github.com/kakao/khaiii.git

!pip install cmake

!mkdir build

!cd build && cmake /content/khaiii

!cd /content/build/ && make all

!cd /content/build/ && make resource

!cd /content/build && make install

!cd /content/build && make package_python

!pip install /content/build/package_python


fatal: destination path 'khaiii' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
-- [khaiii] fused multiply add option enabled
-- [hunter] Calculating Toolchain-SHA1
-- [hunter] Calculating Config-SHA1
-- [hunter] HUNTER_ROOT: /root/.hunter
-- [hunter] [ Hunter-ID: 70287b1 | Toolchain-ID: 02ccb06 | Config-ID: dffbc08 ]
-- [hunter] BOOST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.68.0-p1)
-- Boost version: 1.68.0
-- [hunter] CXXOPTS_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 2.1.1-pre)
-- [hunter] EIGEN_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.5)
-- [hunter] FMT_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 4.1.0)
-- [hunter] GTEST_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 1.8.0-hunter-p11)
-- [hunter] NLOHMANN_JSON_ROOT: /root/.hunter/_Base/70287b1/02ccb06/dffbc08/Install (ver.: 3.3.0)
-- [hunter] SPDLOG_ROOT: /root/.hunte

In [9]:
from khaiii import KhaiiiApi
api = KhaiiiApi()
for word in api.analyze("이거 되냐? 되ㅐ냐고"):
    for morph in word.morphs:
        print(morph.lex)


이거
되
냐
?
되
ㅐ
냐
고


In [10]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torch.nn.functional as F
import torch.nn as nn
#주요 참고 PyTorch로 시작하는 딥 러닝 입문, 유원준
from torchtext import data  
import urllib.request
import pandas as pd
import random

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
def isAll0(x):
    if type(x) == float:
        return x
    elif len(x) == x.count(' '):
        return ''
    else:
        return x


In [13]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")


train_df = pd.read_table('ratings_train.txt')
test_df = pd.read_table('ratings_test.txt')


train_df['document'] = train_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_df['document'] = train_df['document'].apply(isAll0)
train_df['document'].replace('', np.nan, inplace=True)


test_df['document'] = test_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_df['document'] = test_df['document'].apply(isAll0)
test_df['document'].replace('', np.nan, inplace=True)


train_df.drop_duplicates(subset=['document'], inplace=True)
test_df.drop_duplicates(subset=['document'], inplace=True)

train_df = train_df.dropna(how = 'any')
test_df = test_df.dropna(how = 'any')

In [14]:
print(len(train_df))
print(len(test_df))

143660
48403


In [15]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [28]:
train_x = []

In [29]:
j = 0
for i, row in train_df.iterrows():
    val = row['document']
    j += 1
    if j % 10000 == 0:
        print(i,"/","143660")

    temp = ['CLS']
    for word in api.analyze(val):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                temp.append(morph.lex)
    temp.append('SEP')
    train_x.append(temp)


10180 / 143660
20506 / 143660
30877 / 143660
41284 / 143660
51710 / 143660
62159 / 143660
72591 / 143660
83093 / 143660
93557 / 143660
104038 / 143660
114525 / 143660
125029 / 143660
135592 / 143660
146158 / 143660


In [30]:
test_x = []

In [31]:
j = 0
for i, row in test_df.iterrows():
    val = row['document']
    j += 1
    if i % 10000 == 0:
        print(j,"/","48000")
    temp = ['CLS']
    
    for word in api.analyze(val):
        for morph in word.morphs:
            if morph.lex not in stopwords:
                temp.append(morph.lex)
    temp.append('SEP')
    test_x.append(temp)


1 / 48000
9768 / 48000
19467 / 48000
29133 / 48000
38806 / 48000


In [32]:
vocab_size = 10000

tokenizer = Tokenizer(vocab_size,oov_token = 'OOV')
tokenizer.fit_on_texts(train_x)

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

In [33]:
train_y = np.array(train_df['label'])
test_y = np.array(test_df['label'])

In [34]:
drop_train = [index for index, sentence in enumerate(train_x) if len(sentence) < 1]
drop_test = [index for index, sentence in enumerate(test_x) if len(sentence) < 1]

In [35]:
train_x = np.delete(train_x, drop_train, axis=0)
train_y = np.delete(train_y, drop_train, axis=0)
print(len(train_x))
print(len(train_y))

143660
143660


In [36]:
test_x = np.delete(test_x, drop_test, axis=0)
test_y = np.delete(test_y, drop_test, axis=0)
print(len(test_x))
print(len(test_y))

48403
48403


In [37]:
print(train_x)

[list([2, 21, 705, 122, 124, 307, 1540, 74, 757, 3])
 list([2, 1, 9, 8, 718, 1, 773, 675, 12, 43, 384, 3])
 list([2, 30, 178, 1, 11, 6, 1320, 150, 9, 18, 10, 278, 4, 42, 3]) ...
 list([2, 78, 54, 74, 1363, 1, 8, 1, 1, 1058, 6, 3])
 list([2, 1019, 7, 79, 1, 1068, 4, 31, 56, 553, 7664, 380, 3])
 list([2, 177, 7, 1708, 27, 1, 4, 94, 1288, 5, 7, 3])]


In [38]:
pad_len = 30
train_x = pad_sequences(train_x, maxlen = pad_len)
test_X = pad_sequences(test_x, maxlen = pad_len)

In [39]:
print(train_x)

[[   0    0    0 ...   74  757    3]
 [   0    0    0 ...   43  384    3]
 [   0    0    0 ...    4   42    3]
 ...
 [   0    0    0 ... 1058    6    3]
 [   0    0    0 ... 7664  380    3]
 [   0    0    0 ...    5    7    3]]
